In [48]:
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer


from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier


from sklearn.metrics import confusion_matrix
from GetScoreSepsis import GetScoreSepsis

# Načtení dat

In [49]:
# Cesta k tabulce s pacienty
filepath = './dataSepsis.csv'

# Načtení celé tabulky
fullSepsis = pd.read_csv(filepath, sep=';')

# Odstranění sloupců a vymazání řádků s příliš mnoha NaN

In [50]:
# Seznam všech sloupců, které mají být odstraněny
# Sloupce jsem vybírali na základě počtu NaN, korelace a statistické významnosti

drop_list = ['Temp','EtCO2','BaseExcess','HCO3','FiO2','pH',
 'PaCO2','SaO2','AST','Alkalinephos','Chloride','Bilirubin_direct',
 'Lactate','Phosphate','Bilirubin_total','TroponinI','PTT','Fibrinogen','Unit1',
 'Unit2', 'SBP', 'DBP', 'Hct', 'Age', 'Platelets', 'BUN']

# Odstranění sloupců
relevantSepsis = fullSepsis.drop(columns=drop_list)

# Odstranění řádků, které mají polovinu a více hodnot jako NaN hodnoty
relevantSepsis = relevantSepsis.dropna(thresh=relevantSepsis.shape[1]/2)
relevantSepsis = relevantSepsis.reset_index()
relevantSepsis.drop(columns=['index'], inplace=True)

# Separování sloupce s určením, zda je pacient sepický či nikoliv

In [ ]:
isSepsis = relevantSepsis["isSepsis"]
relevantSepsis.drop(columns=["isSepsis"])

# Předzpracování dat

In [51]:
df = relevantSepsis
# Výpočet interquartilového rozptylu pro každý sloupce
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Stanovení horní a dolní hranice
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Nahrazení odlehlých hodnot hodnotami NaN na základě daných hranic
df_outliers_removed = df[(df >= lower_bound) & (df <= upper_bound)]

# Standardizace datasetu (dojde k převedení na array)
scaler = StandardScaler()
standardizedSepsis = scaler.fit_transform(df_outliers_removed)

# Nahrazení NaN pomocí k-nejbližších sousedů
imputer = KNNImputer(n_neighbors=100)
finalSepsis = imputer.fit_transform(standardizedSepsis)

# Zpětné převedení na dataframe
preprocessed_data = pd.DataFrame(data= finalSepsis, columns=df.columns)

# Vytvoření dvou datasetů s náhodně vybranými pacienty se sepsí a bez sepse

In [62]:
sepsis_df = preprocessed_data[isSepsis==1].sample(2000)
no_sepsis_df = preprocessed_data[isSepsis==0].sample(2000)

# Trénování modelu

In [ ]:
# Zpětné přidání informace o diagnostice sepse
sepsis_df['sepsis'] = 1
no_sepsis_df['sepsis'] = 0

# Spojení datasetů do testovacího datasetu
combined_df = pd.concat([sepsis_df, no_sepsis_df], ignore_index=True)

# Oddělení příznaků (X) od předpovídané proměnné (y)
X = combined_df.drop('sepsis', axis=1)
y = combined_df['sepsis']

# Rozdělení dat na testovací a trénovací množinu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Vytvoření XGBoost classifier
model = XGBClassifier(n_estimators=100,
                      max_depth=4,
                      max_leaves=4,
                      grow_policy="depthwise",
                      booster="gbtree",
                      early_stopping_rounds=10
                      )

model.fit(X_train, y_train, eval_set=[(X_test, y_test)])


# Zjišťování score pro trénovací množinu

In [64]:
yPredTrain = model.predict(X_train)

confMatrixTrain = confusion_matrix(y_train, yPredTrain)
GetScoreSepsis(confMatrixTrain)

(0.867877269881027,
 0.8739862757330006,
 0.8709375,
 0.8727959697732998,
 0.8703296703296703)

# Zjišťování score pro testovací množinu

In [65]:
yPred = model.predict(X_test)

confMatrix = confusion_matrix(y_test, yPred)
GetScoreSepsis(confMatrix)

(0.8216867469879519,
 0.8155844155844156,
 0.81875,
 0.8276699029126213,
 0.8246674727932285)

# Uložení modelu

In [47]:
joblib.dump(model, "HVH_model.joblib")

['HVH_model.joblib']